In [ ]:
# import libs
import pandas as pd
import pprint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# get titles from excels
def get_records_from_excel(file_path):
    df = pd.read_excel(file_path)
    values_set = list(df.iloc[:, 0])
    return values_set


records = get_records_from_excel("/content/drive/My Drive/project/finals/output.xlsx")
print(records)
records2 = get_records_from_excel("/content/drive/My Drive/project/finals/output2.xlsx")
print(records2)

['شارژر همراه 20000 میلی آمپر شیائومی مدل Redmi PB200LZM همراه با 2 پورت خروجی', 'شارژر همراه 10000 میلی آمپر شیائومی مدل Redmi PB100LZM همراه با 2 پورت خروجی', 'کیبورد و ماوس بی سیم شیائومی مدل WXJS01YM', 'مانیتور 19 اینچی داهوا مدل DHI-LM19-A200', 'کابل HDMI آر تی سی مدل RT17HD طول 17 متر', 'کارت حافظه microSDHC ویکو من مدل Extre600X کلاس 10 سرعت 90MBps ظرفیت 32 گیگابایت همراه با آداپتور SD', 'کارت هدیه 100.000 تومانی تیمچه طرح دوستت دارم', 'مانیتور 24 اینچی ایسوس مدل VG24VQ با کیفیت تصویر FULL HD', 'تبلت 10.1 اینچی سامسونگ مدل Galaxy TAB A 2019 LTE SM-T515 با ظرفیت 32 گیگابایت', 'تبلت 11.0 اینچی سامسونگ مدل Galaxy Tab S7 SM-T875 با ظرفیت 128 گیگابایت', 'مودم 4G/LTE قابل حمل بی سیم دی-لینک مدل DWR-910M', 'کیبورد و ماوس مخصوص بازی وریتی مدل V-KB6121GCW', 'فلش مموری سن دیسک مدل ULTRA FIT USB 3.1 CZ430 ظرفیت 64 گیگابایت با رابط USB 3.1', 'کابل HDMI مدل NV-014 طول 1.5 متر', 'شارژر همراه 10000 میلی آمپر یونیوو مدل UN10 همراه با 2 پورت خروجی', 'مانیتور 27 اینچی شیائومی مدل RMMNT27NF با کیف

In [ ]:
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 24.1 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=19d86b207febd7c032c1f92a67d4c483176cdfbba7984b9961707f992bf4bab8
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [ ]:
# import stopwords
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from hazm import stopwords_list

# خواندن کلمات فارسی از فایل persion.txt
# with open('/content/drive/My Drive/project/finals/persian.txt', 'r', encoding='utf-8') as file:
#     persian_stopwords = file.read().split()

english_stopwords = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from hazm import word_tokenize
from hazm import Normalizer
from hazm import Lemmatizer

# Function to process sentences
def process_sentences(records):
    normalizer = Normalizer()
    lemmatizer = Lemmatizer()
    processed_sentences = []

    for sentence in records:
        normalized_sentence = normalizer.normalize(sentence)
        tokens = word_tokenize(normalized_sentence)
        processed_tokens = []

        for word in tokens:
            if re.search(r'[\u0600-\u06FF]', word):  # Persian
                word = lemmatizer.lemmatize(word)
                if word not in stopwords_list():
                    processed_tokens.append(word)
            elif word.lower() not in english_stopwords:  # English
                processed_tokens.append(word)

        processed_sentences.append(" ".join(processed_tokens))

    return processed_sentences


# Process sentences
dic1 = process_sentences(records)
dic2 = process_sentences(records2)



```
TFIDF
```



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Function to calculate cosine similarity
def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)[0][1]
    return cosine_sim

# Find the most similar record using cosine similarity and store results in a DataFrame
results = []

for str1 in dic1:
    max_similarity = 0
    most_similar_text = ""
    for str2 in dic2:
        similarity_score = calculate_cosine_similarity(str1, str2)
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            most_similar_text = str2
    results.append([str1, most_similar_text, max_similarity])

# Create a DataFrame
df_results = pd.DataFrame(results, columns=['Record from Table 1', 'Most Similar Record from Table 2', 'Similarity'])

# Display the DataFrame
df_results.head()

,Record from Table 1,Most Similar Record from Table 2,Similarity
0,شارژر همراه ۲۰۰۰۰ میلی‌آمپر شیائومی مدل Redmi ...,شارژر همراه ۲۰۰۰۰ میلی‌آمپر شیائومی مدل MI - R...,0.895532
1,شارژر همراه ۱۰۰۰۰ میلی‌آمپر شیائومی مدل Redmi ...,شارژر همراه ۲۰۰۰۰ میلی‌آمپر شیائومی مدل MI - R...,0.706078
2,کیبورد ماوس بی‌سیم شیائومی مدل WXJS ۰۱ YM,کیبورد سیم دل مدل Rugged ۷۲۰۲,0.260556
3,مانیتور ۱۹ اینچ داهوا مدل DHI-LM ۱۹ -A ۲۰۰,مانیتور ۲۴ اینچ جی پلاس مدل GDM- ۲۴۵ LN کیفیت ...,0.145040
4,کابل HDMI آر تی مدل RT ۱۷ HD ۱۷,کابل HDMI آر تی مدل RT ۱۷ HD ۱۷,1.000000




```
Bert
```



In [ ]:
from transformers import TFBertModel, BertTokenizer
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

# download BERT model و tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture

In [ ]:
# vectorize sentences
def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='tf', padding=True, truncation=True)
    outputs = model(inputs)
    embeddings = tf.reduce_mean(outputs.last_hidden_state, axis=1)
    return embeddings

# caclute similarity with cosine similarity
def calculate_similarity(embedding1, embedding2):
    embedding1 = embedding1.numpy()
    embedding2 = embedding2.numpy()
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]

In [ ]:
# find most similar one
def find_most_similar_records(list1, list2):
    similarity_results = []

    for idx1, sentence1 in enumerate(list1):
        max_similarity = -1
        most_similar_idx = -1
        embedding1 = get_embedding(sentence1)

        for idx2, sentence2 in enumerate(list2):
            embedding2 = get_embedding(sentence2)
            similarity = calculate_similarity(embedding1, embedding2)

            if similarity > max_similarity:
                max_similarity = similarity
                most_similar_idx = idx2

        similarity_results.append((records[idx1], records2[most_similar_idx], max_similarity))
    return similarity_results

similarity_results = find_most_similar_records(dic1, dic2)
df_results = pd.DataFrame(similarity_results, columns=['Record from List 1', 'Most Similar Record from List 2', 'Similarity'])

# Display the results
df_results.head()

,Record from List 1,Most Similar Record from List 2,Similarity
0,شارژر همراه 20000 میلی آمپر شیائومی مدل Redmi ...,شارژر همراه 20000 میلی آمپر شیائومی مدل MI - R...,0.989339
1,شارژر همراه 10000 میلی آمپر شیائومی مدل Redmi ...,شارژر همراه 20000 میلی آمپر شیائومی مدل MI - R...,0.976073
2,کیبورد و ماوس بی سیم شیائومی مدل WXJS01YM,کیبورد با سیم دل مدل Rugged 7202,0.819152
3,مانیتور 19 اینچی داهوا مدل DHI-LM19-A200,شارژر همراه 20000 میلی آمپر شیائومی مدل MI - R...,0.845399
4,کابل HDMI آر تی سی مدل RT17HD طول 17 متر,کابل HDMI آر تی سی مدل RT17HD طول 17 متر,1.000000
